<h1> <center> METRO MANILA: COMMODITY PRICE CHECKER </center> </h1>

<h1 style="background-color: #3cb371; padding: 20px;"> <center> <span style="color:white">
<br> INSTALLATION
</span> </center> </h1>

In [1]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import itertools
import logging
import os

import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

pd.set_option('display.max_rows', None)

# Helper function: Prompt user input
def prompt_choice(prompt, options):
    print(prompt)
    for key, value in options.items():
        print(f"[{key}] {value}")
    while True:
        try:
            choice = int(input("Choose an option: "))
            if choice in options:
                return options[choice]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Define commodity mappings
COMMODITY_GROUPS = {
    1: ("Rice", {
        1: ('Regular-Milled Rice', '../csv/rice/regular_milled_rice.csv'),
        2: ('Well-Milled Rice', '../csv/rice/well_milled_rice.csv'),
        3: ('Premium Rice', '../csv/rice/premium_rice.csv'),
        4: ('Special Rice', '../csv/rice/special_rice.csv'),
    }),
    2: ("Meat", {
        1: ('Beef Brisket', '../csv/meat/beef_brisket.csv'),
        2: ('Beef Rump', '../csv/meat/beef_rump.csv'),
        3: ('Whole Chicken', '../csv/meat/whole_chicken.csv'),
        4: ('Pork Belly', '../csv/meat/pork_belly.csv'),
        5: ('Pork Ham', '../csv/meat/pork_kasim.csv'),
    }),
    3: ("Fish", {
        1: ('Alumahan', '../csv/fish/alumahan.csv'),
        2: ('Bangus', '../csv/fish/bangus.csv'),
        3: ('Galunggong', '../csv/fish/galunggong.csv'),
        4: ('Tilapia', '../csv/fish/tilapia.csv'),
    }),
    4: ("Fruits", {
        1: ('Banana (Lakatan)', '../csv/fruits/banana_lakatan.csv'),
        2: ('Calamansi', '../csv/fruits/calamansi.csv'),
        3: ('Mango', '../csv/fruits/mango.csv'),
        4: ('Papaya', '../csv/fruits/papaya.csv'),
    }),
    5: ("Vegetables", {
        1: ('Cabbage', '../csv/vegetables/cabbage.csv'),
        2: ('Carrots', '../csv/vegetables/carrots.csv'),
        3: ('Eggplant', '../csv/vegetables/eggplant.csv'),
        4: ('Tomato', '../csv/vegetables/tomato.csv'),
        5: ('Potato', '../csv/vegetables/white_potato.csv'),
    }),
    6: ("Spices", {
        1: ('Garlic', '../csv/spices/garlic.csv'),
        2: ('Red Onion', '../csv/spices/onion.csv'),
    })
}

# Grid search for ARIMA parameters
def grid_search_arima(df, column, p_range=(0, 5), d_range=(0, 3), q_range=(0, 5)):
    data = df[column].dropna()
    best_score, best_params = float('inf'), None

    for p, d, q in itertools.product(range(*p_range), range(*d_range), range(*q_range)):
        try:
            model = ARIMA(data, order=(p, d, q)).fit()
            predictions = model.fittedvalues
            mse = mean_squared_error(data[d:], predictions[d:])
            if mse < best_score:
                best_score, best_params = mse, (p, d, q)
        except Exception:
            continue

    logging.info(f"Best Parameters: {best_params} with MSE: {best_score}")
    return best_params, best_score

# Train ARIMA model
def train_arima_model(df, column, params):
    model = ARIMA(df[column], order=params).fit()
    logging.info(f"Model Summary: \n{model.summary()}")
    return model

# Main function
def main():
    # Choose commodity group
    group_choice = prompt_choice("Select a commodity group:", {k: v[0] for k, v in COMMODITY_GROUPS.items()})
    if not group_choice:
        logging.error("Invalid commodity group selected.")
        return

    # Map back to the correct key from the selected group
    commodity_group_key = next((k for k, v in COMMODITY_GROUPS.items() if v[0] == group_choice), None)
    if commodity_group_key is None:
        logging.error("Invalid commodity group mapping.")
        return

    # Choose specific commodity
    commodity_details = prompt_choice("Select a specific commodity:", COMMODITY_GROUPS[commodity_group_key][1])
    if not commodity_details:
        logging.error("Invalid commodity selected.")
        return

    commodity, csv_path = commodity_details
    logging.info(f"You have selected {commodity}.")

    # Check if file exists
    if not os.path.exists(csv_path):
        logging.error(f"File not found: {csv_path}")
        return

    # Load data
    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
    logging.info("Data loaded successfully.")

    # Check if required columns exist
    for column in ["High", "Low"]:
        if column not in df.columns:
            logging.warning(f"Column '{column}' not found in the dataset. Skipping analysis for this column.")
            continue

        logging.info(f"Analyzing {column} prices.")
        params, mse = grid_search_arima(df, column)
        if params:
            logging.info(f"Optimal parameters for {column}: {params} with MSE: {mse}")
        else:
            logging.warning(f"No valid parameters found for {column}.")

# Run the application
if __name__ == "__main__":
    main()